# COMP0005 - GROUP COURSEWORK 2023-24
# Gesture Recognition via Convex Hull 

In [186]:
from math import pi, atan2 # for computing polar angle

# This class contains utility functions for graph operations
class GraphUtils():
    
    # This function returns the cross product which calculates the area of the parallelogram of the line drawn from 'a' to 'b' to 'c'.
    def orientation(self, a, b, c):
        val = (b[1] - a[1]) * (c[0] - b[0]) - (b[0] - a[0]) * (c[1] - b[1])
        # return 0 if collinear
        if val == 0:
            return 0
        # return 1 if counterclockwise orientation
        elif val > 0:
            return 1
        # return -1 if clockwise orientation
        else:
            return -1
    
    # This function returns the point with the minimum y-coordinate. If there are multiple points with the same y-coordinate, 
    # it returns the point with the minimum x-coordinate.
    def getMinY(self, points):
        min = points[0]
        for point in points:
            if point[1] < min[1]:
                min = point
            elif point[1] == min[1]:
                if point[0] < min[0]:
                    min = point
        return min
    
    # This function returns the distance between two points.
    def dist(self, a, b):
        xDiffSquare = (a[0] - b[0]) ** 2
        yDiffSquare = (a[1] - b[1]) ** 2
        dist = (xDiffSquare + yDiffSquare) ** (1/2)
        return dist
    
    # This function returns the 2 argument arc tangent of the point 'b' with respect to the point 'a'.
    def angle(self, a, b):
        angle = atan2(b[1] - a[1], b[0] - a[0])
        if angle < 0:
            angle += 2 * pi
        return angle

Use the cell below for all python code needed to realise the **Jarvis march algorithm** (including auxiliary data structures and functions needed by this algorithm - if any). The `jarvismarch()` function itself should take as input parameter a list of 2D points (`inputSet`), and return the subset of such points (`outputSet`) that lie on the convex hull.

In [187]:
import math

def jarvismarch(inputSet):
    '''
    Returns the list of points that lie on the convex hull (jarvis march algorithm)
            Parameters:
                    inputSet (list): a list of 2D points

            Returns:
                    outputSet (list): a list of 2D points
    '''
    
    graphUtils = GraphUtils()

    #if less than 3 points in set, all point in set guarenteed to be on the convex hull
    if len(inputSet) <= 3:
        return inputSet
    
    #start with the first element in the set
    lCoord = inputSet[0][0]
    lPt = inputSet[0]
    #find leftmost point in the set
    for i in range(len(inputSet)):
        if inputSet[i][0] < lCoord or (inputSet[i][0] == lCoord and inputSet[i][1] > lPt[1]):
            lCoord = inputSet[i][0]
            lPt = inputSet[i]
    outputSet = [lPt]
    #set the start point to be the current point
    nextPt = lPt

    while True: 
        #pick the first point in set to be the next possible point on convex hull
        if inputSet[0] != nextPt:
            ref = inputSet[0]
        else:
            ref= inputSet[1]

        #iterate through set to compaqre other point with the point referenced by ref
        for pt in inputSet:
            #go to next iteration if the point is the current point
            if pt == nextPt:
                continue
            #calculate cross product to find out the relationship between points
            orientation = graphUtils.orientation(nextPt, ref, pt)
            #if the points are found to be clockwise or if they are collinear and the point selected to make comparison with the 
            #ref point is further to the current point than the ref point, swap the point with the ref point
            if(orientation < 0) or (orientation == 0 and (graphUtils.dist(nextPt,pt))**2 > (graphUtils.dist(nextPt,ref))**2):
                ref = pt
            
        nextPt = ref

        #if the current point is back to the start point, end the loop
        if nextPt == lPt:
            break

        outputSet.append(nextPt)

    return outputSet

Use the cell below for all python code needed to realise the **Graham scan** algorithm (including auxiliary data structures and functions needed by this algorithm - if any). The `grahamscan()` function itself should take as input parameter a list of 2D points (`inputSet`), and return the subset of such points that lie on the convex hull (`outputSet`).

In [188]:
def grahamscan(inputSet):
    '''
    Returns the list of points that lie on the convex hull (graham scan algorithm)
            Parameters:
                    inputSet (list): a list of 2D points

            Returns:
                    outputSet (list): a list of 2D points
    '''

    graphUtils = GraphUtils()       
    #if the inputSet is relatively small, the convex hull is the input set itself
    if len(inputSet) <= 3:
        return inputSet

    #defines the sorting key based on the tuple consisting of polar angles and disntace from the pivot
    def sort_key(point, P):
        angle = atan2(point[1] - P[1], point[0] - P[0])
        distanceSquared = (point[0] - P[0])**2 + (point[1] - P[1])**2
        return (angle, distanceSquared)
    
    outputSet = [] #outputSet acts as a stack
    minYPoint = graphUtils.getMinY(inputSet) #gets the min y co-ord of points in input set. if equal, gets the minimum x co-ord of the equal points
    inputSet.remove(minYPoint)
    #sorts the remaining points as above, and adds the pivot point and the 1st point from the sorted list to the stack
    sortedInputSet = sorted(inputSet, key=lambda p: sort_key(p, minYPoint))
    outputSet.append(minYPoint)
    outputSet.append(sortedInputSet[0])

    #iterates through the sorted list of points, and adds the point to the stack if it is a part of the convex hull
    for i in range(1, len(sortedInputSet)):
        next = sortedInputSet[i]
        #check for non-left turns; pop the last point from the stack if it's a right turn or collinear
        while len(outputSet) > 1 and graphUtils.orientation(outputSet[-2], outputSet[-1], next) != -1:
            outputSet.pop()
        outputSet.append(next) #add point to stack once a left turn is confirmed

    return outputSet #contains the points on the convex hull

Use the cell below for all python code needed to realise the **Chen's** algorithm (including auxiliary data structures and functions needed by this algorithm - if any). The `chen()` function itself should take as input parameter a list of 2D points (`inputSet`), and return the subset of such points that lie on the convex hull (`outputSet`).

In [189]:
def chen(inputSet):
    '''
    Returns the list of points that lie on the convex hull (chen's algorithm)
            Parameters:
                    inputSet (list): a list of 2D points

            Returns:
                    outputSet (list): a list of 2D points
    '''

    graphUtils = GraphUtils()  
    
     # if the inputSet is relatively small, the convex hull is the input set itself
    if len(inputSet) <= 3:
          return inputSet
    
    # This function computes the tangent from a point 'p' to a convex 'hull' by maximizing the angle made by the point 'p' with the points on the hull. 
    # Returns the index of the point in the inout 'hull'.
    def tangent(hull, p):
          l = 0 #left most element in the list
          r = len(hull) #right most element in the list
          lPre = graphUtils.orientation(p, hull[0], hull[-1])
          lSucc = graphUtils.orientation(p, hull[0], hull[(l + 1) % r])
          while l < r:
               m = (l + r) // 2
               mPre = graphUtils.orientation(p, hull[(m - 1) % len(hull)], hull[m])
               mSucc = graphUtils.orientation(p, hull[m], hull[(m + 1) % len(hull)])
               mSide = graphUtils.orientation(p, hull[l], hull[m])
               if mPre == 1 and mSucc != 1:
                    return m
               elif (mSide == -1 and (lSucc == 1 or lPre == lSucc)) or (mSide == 1 and mPre == -1):
                    r = m
               else:
                    l = m + 1           
                    lPre = -mSucc    
                    if l < len(hull):
                         lSucc = graphUtils.orientation(p, hull[l], hull[(l + 1) % len(hull)])
                    else: 
                         return -1
          return l
     
    # Returns the index of the hull with the minimum y-coordinate.
    def minHull(hulls):
          minHull = 0
          for i in range(len(hulls)):
               if hulls[i][0][1] < hulls[minHull][0][1] or (hulls[i][0][1] == hulls[minHull][0][1] and hulls[i][0][0] < hulls[minHull][0][0]):
                    minHull = i
          return (minHull)

    # Finds the next point pair in the convex hull construction.
    # This function selects the next point on the convex hull from the set of mini-hulls.
    # It uses the tangent function to find the most outward point in the direction of hull construction.
    def nextHullPtPair(hulls, pair):
          p = hulls[pair[0]][pair[1]]
          nextPair = (pair[0], (pair[1] + 1) % len(hulls[pair[0]]))
          for i in range(len(hulls)):
               if i == pair[0]:
                    continue
               s = tangent(hulls[i], p)
               q = hulls[nextPair[0]][nextPair[1]]
               if s < len(hulls[i]):
                    r = hulls[i][s]
                    t = graphUtils.orientation(p,q,r)
                    if t == 1 or (t == 0 and graphUtils.dist(p, r) > graphUtils.dist(p,q)):
                         nextPair = (i, s)
          return nextPair

    # Main loop of Chen's algorithm
    t = round(math.log(math.log(len(inputSet), 2), 2) - 1)
    while True:
         outputSet = []
         m = min(2 ** (2 ** t), len(inputSet))
         hulls = [grahamscan(inputSet[i:i+m]) for i in range(0, len(inputSet), m)]
         leftMostHull = minHull(hulls)
         currentHullPtPair = (leftMostHull, 0)
         outputSet.append(hulls[currentHullPtPair[0]][currentHullPtPair[1]])
         
         # Iteratively add points to the convex hull.
         for _ in range(m):
              (hullIndex, ptIndex) = nextHullPtPair(hulls, currentHullPtPair)
              currentHullPtPair = (hullIndex, ptIndex)
              if hulls[hullIndex][ptIndex] == outputSet[0]:
                   return outputSet
              outputSet.append(hulls[hullIndex][ptIndex])
         t+=1


Use the cell below to implement the **synthetic data generator** needed by your experimental framework (including any auxiliary data structures and functions you might need - be mindful of code readability and reusability).

In [190]:
import random

class TestDataGenerator():
    """
    A class to represent a synthetic data generator.
    """
        
    #ADD YOUR CODE HERE
    
    def __init__(self, maxXAndY):
        self.maxXAndY = maxXAndY
        self.radius = maxXAndY // 2
        self.points = []

    #this function generates random points within the range of X and Y
    def randomPoints(self,n):
        points = []
        for _ in range(n):
            points.append((random.randint(0,self.maxXAndY - 1),random.randint(0,self.maxXAndY - 1)))
        self.points = points

    #this function generates points on the edge of a polygon. This is limited to 600 as after 600, the function does not give n == h which is what we need the function to do.
    def pointsOnPolygon(self, h):
        radius = self.radius
        points = []
        for i in range(h):
            angle = 2 * math.pi * i/h
            x = round(radius + radius * math.cos(angle))
            y = round(radius + radius * math.sin(angle))
            points.append((x, y))
        self.points = points 
    
    #this function returns a bool if a point is within the polygon formed by the points in the polyPoints
    def isPointInsidePolygon(self, x, y, polyPoints):
        n = len(polyPoints)
        inside = False
        xPoint1, yPoint1 = polyPoints[0]
        
        for i in range(n + 1):
            xPoint2, yPoint2 = polyPoints[i % n]
            if y > min(yPoint1, yPoint2):
                if y <= max(yPoint1, yPoint2):
                    if x <= max(xPoint1, xPoint2):
                        if yPoint1 != yPoint2:
                            xints = (y - yPoint1) * (xPoint2 - xPoint1) / (yPoint2 - yPoint1) + xPoint1
                            if xPoint1 == xPoint2 or x <= xints:
                                inside = not inside
            xPoint1, yPoint1 = xPoint2, yPoint2
        
        return inside

    #this function uses the above 2 functions to generate a polygon with h sides and a total of n points. This is used to vary h and keep n constant. However, this is also limited to a h value of 600 
    def polygonPoints(self, n, h):
        pointsInsidePolygon = n - h
        self.pointsOnPolygon(h)
        vertices = self.points

        sumX = sum([x for x,y in vertices])
        centroidX = sumX / h
        sumY = sum([y for x,y in vertices])
        centroidY = sumY / h
        centroid = (centroidX,centroidY)

        points = set()
        while len(points) < pointsInsidePolygon:
            angle = 2 * math.pi * random.random()
            radius = self.radius * math.sqrt(random.random())
            x = centroid[0] + radius * math.cos(angle)
            y = centroid[1] + radius * math.sin(angle)
            
            x = int(round(x))
            y = int(round(y))

            if self.isPointInsidePolygon(x, y, vertices):
                points.add((x, y))
        points = list(points)
        points += vertices
        self.points = points 

    #this function generates 4 corner points and random points within these corners to test if the algorithms take into account colinear points.
    def pointsSquare(self,n, h):
        
        self.randomPoints(n - h - 4)
        hOneSide = h // 4
        points = []
        for _ in range(hOneSide):
            points.append((0,random.randint(0,self.maxXAndY - 1)))
        for _ in range(hOneSide):
            points.append((self.maxXAndY,random.randint(0,self.maxXAndY - 1)))
        for _ in range(hOneSide):
            points.append((random.randint(0,self.maxXAndY - 1),0))
        for _ in range(hOneSide):
            points.append((random.randint(0,self.maxXAndY - 1),self.maxXAndY))
        points.append((0,0))
        points.append((0,self.maxXAndY))
        points.append((self.maxXAndY, self.maxXAndY))
        points.append((self.maxXAndY, 0))
        self.points = self.points + points


Use the cell below to implement the requested **experimental framework** API.

In [191]:
import timeit
import matplotlib.pyplot as plt

class ExperimentalFramework():
    """
    A class to represent an experimental framework.

    """
            
    def __init__(self):
        self.dataGenerator = TestDataGenerator(32767)
        self.timingsjm = []
        self.timingsgs = []
        self.timingsc = []
        self.jm = []
        self.gs = []
        self.c =[]

    #this function tests for colinear points in the algorihtms
    def colinearTest(self, algo, n, h):
        self.dataGenerator.pointsSquare(n,h)
        points = self.dataGenerator.points
        x = [a for a,b in points]
        y = [b for a,b in points]
        plt.scatter(x,y)

        hullPoints = algo(self.dataGenerator.points)
        xHull = [aHull for aHull,bHull in hullPoints]
        yHull = [bHull for aHull,bHull in hullPoints]
        xHull.append(hullPoints[0][0])
        yHull.append(hullPoints[0][1])
        plt.plot(xHull,yHull)

        if len(hullPoints) == 4:
            print("Colinear Test is True")
        else:
            print("Colinear Test is False")


    #these next function in this class calculate the execution time of the algorithms and the functions that plot the graphs.
        
    #this the function that calculates the execution time for one run of the 'algo' algorithm on the 'inputSet'.
    def testTiming(self, algo,inputSet):
        start_time = timeit.default_timer()
        algo(inputSet)
        return timeit.default_timer() - start_time

    #all the graph functions test the same x-axis values, with random points, 9 times and then get the median value by sorting the list and then using the element at index 4. This is done to account for outliers.
    
    #in the graph functions, the maximum x-axis value is the 'maxn' or 'maxh' and 'incre' is increment it takes while testing the function(not the increment on the x-axis of the graph)
    
    #this function plots the graph for the average case scenario of all 3 algorithms 
    def plotGraph1(self,maxn ,incre):

        self.timingsjm = []
        self.timingsgs = []
        self.timingsc = []
        self.jm = []
        self.gs = []
        self.c =[]
        for n in range(3, maxn + 4, incre): 
            for _ in range(9):
                self.dataGenerator.randomPoints(n)
                self.jm.append(self.testTiming(jarvismarch, self.dataGenerator.points))
                self.gs.append(self.testTiming(grahamscan, self.dataGenerator.points))
                self.c.append(self.testTiming(chen, self.dataGenerator.points))
            
            self.jm.sort()
            self.gs.sort()
            self.c.sort()
            self.timingsjm.append(self.jm[4])
            self.timingsgs.append(self.gs[4])
            self.timingsc .append(self.c[4])
            self.jm.clear()
            self.gs.clear()
            self.c.clear()

    
        plt.figure(1)
        plt.title("Average Case Scenario")
        plt.xlabel("Number of Points")
        plt.ylabel("Average Execution Time (seconds)")
        plt.grid(True)

        plt.plot(range(3, maxn + 4, incre), self.timingsjm, color = "black", label = "Jarvis March")
        plt.plot(range(3, maxn + 4, incre), self.timingsgs, color = "blue", label = "Graham's Scan")
        plt.plot(range(3, maxn + 4, incre), self.timingsc, color = "orange", label = "Chan's Algorithm")
        plt.legend()


    #this function plots the graph for the worst case scenario of jarvismarch and chan's algorithms. This case is the best case scenario for grahamscan. 
    def plotGraph2(self, maxn , incre):

        self.timingsjm = []
        self.timingsgs = []
        self.timingsc = []
        self.jm = []
        self.gs = []
        self.c =[]
        for n in range(3, maxn + 1, incre): 
            for _ in range(9):
                self.dataGenerator.pointsOnPolygon(n)
                self.jm.append(self.testTiming(jarvismarch, self.dataGenerator.points))
                self.gs.append(self.testTiming(grahamscan, self.dataGenerator.points))
                self.c.append(self.testTiming(chen, self.dataGenerator.points))
            
            self.jm.sort()
            self.gs.sort()
            self.c.sort()
            self.timingsjm.append(self.jm[4])
            self.timingsgs.append(self.gs[4])
            self.timingsc .append(self.c[4])
            self.jm.clear()
            self.gs.clear()
            self.c.clear()


        plt.figure(2)
        plt.title("Worst Case Scenario")
        plt.xlabel("Number of Points")
        plt.ylabel("Average Execution Time (seconds)")
        plt.grid(True)

        plt.plot(range(3, maxn + 1, incre), self.timingsjm, color = "black", label = "Jarvis March")
        plt.plot(range(3, maxn + 1, incre), self.timingsgs, color = "blue", label = "Graham's Scan")
        plt.plot(range(3, maxn + 1, incre), self.timingsc, color = "orange", label = "Chan's Algorithm")
        plt.legend()

    
    #this fucntion plots a graph with a constant n value and varying the h value for all 3 algorithms 
    def plotGraph3(self, n, maxh , incre):

        self.timingsjm = []
        self.timingsgs = []
        self.timingsc = []
        self.jm = []
        self.gs = []
        self.c =[]
        for h in range(3, maxh + 1, incre): 
            for _ in range(9):
                self.dataGenerator.polygonPoints(n,h)
                self.jm.append(self.testTiming(jarvismarch, self.dataGenerator.points))
                self.gs.append(self.testTiming(grahamscan, self.dataGenerator.points))
                self.c.append(self.testTiming(chen, self.dataGenerator.points))
            
            self.jm.sort()
            self.gs.sort()
            self.c.sort()
            self.timingsjm.append(self.jm[4])
            self.timingsgs.append(self.gs[4])
            self.timingsc .append(self.c[4])
            self.jm.clear()
            self.gs.clear()
            self.c.clear()


        plt.figure(3)
        plt.title("'n' Constant and 'h' Varied")
        plt.xlabel("Number of Points on the Hull 'h'")
        plt.ylabel("Average Execution Time (seconds)")
        plt.grid(True)

        plt.plot(range(3, maxh + 1, incre), self.timingsjm, color = "black", label = "Jarvis March")
        plt.plot(range(3, maxh + 1, incre), self.timingsgs, color = "blue", label = "Graham's Scan")
        plt.plot(range(3, maxh + 1, incre), self.timingsc, color = "orange", label = "Chan's Algorithm")
        plt.legend()


    #this function plots a graph for different h/n ratios as n increases an algorithm 
    def plotGraph4(self, algo, maxn, incre, startRatio, ratioIncre): 

        totalRatios = int(((1 - startRatio)/ ratioIncre) + 1)
        timings = [[]]
        for _ in range(totalRatios - 1):
            timings.append([])

        testTimings = []
       
       #starting from 30 so that we can take all ratios starting from 0.1 as 30 * 0.1 = 3 which is the lowest possible value for h
        for n in range(30, maxn + 1, incre):
            ratio = startRatio
            for i in range(totalRatios):
                h = round(n * ratio)
                for _ in range(9):
                    self.dataGenerator.polygonPoints(n,h)
                    testTimings.append(self.testTiming(algo, self.dataGenerator.points))
                testTimings.sort()
                timings[i].append(testTimings[4])
                testTimings.clear()
                ratio += ratioIncre 

        if "jarvismarch" in str(algo):
            algorithm = "Jarvis March"
        elif "grahamscan" in str(algo):
            algorithm = "Graham Scan"
        else: 
            algorithm = "Chan's Algorithm"
        plt.figure(4)
        plt.title("Ratios of h / n Varied for " + algorithm)
        plt.xlabel("Number of Points")
        plt.ylabel("Average Execution Time (seconds)")
        plt.grid(True)

        ratio = startRatio
        for i in range(totalRatios):
            plt.plot(range(30, maxn +1, incre), timings[i], label = "h / n = " + str(ratio))
            ratio += ratioIncre
        plt.legend()


Use the cell below to illustrate the python code you used to **fully evaluate** the three convex hull algortihms under considerations. The code below should illustrate, for example, how you made used of the **TestDataGenerator** class to generate test data of various size and properties; how you instatiated the **ExperimentalFramework** class to  evaluate each algorithm using such data, collect information about their execution time, plots results, etc. Any results you illustrate in the companion PDF report should have been generated using the code below.

In [192]:
eF = ExperimentalFramework()

Colinear Tests

In [ ]:
#Jarvis March
eF.colinearTest(jarvismarch,200,50)

In [ ]:
#Graham Scan
eF.colinearTest(grahamscan,200,50)

In [ ]:
#Chen's Algorithm
eF.colinearTest(chen,200,50)

Average Case Graph for all Three Algorithms

In [ ]:
#maxn = 1 million and increment = 50k
eF.plotGraph1(1000000, 50000)

Worst Case Graph for Jarvis March and Chan's Algorithm and the Best Case for GrahamScan

In [ ]:
#n = h, maxn = 600 and increment = 10
eF.plotGraph2(600, 10)

'n' Constant and 'h' Varied Graph for all Three Algorithms

In [ ]:
#n = 10000 (constant), maxh = 12 and increment = 1
eF.plotGraph3(10000, 12, 1)

Graphs for Four Different 'h' / 'n' Ratios for each Algorithm

In [ ]:
#Jarvis March

#maxn = 600, increment = 10, start ratio = 0.25 and ratio increment = 0.25
eF.plotGraph4(jarvismarch, 600, 10, 0.25, 0.25)

In [ ]:
#Graham's Scan

#maxn = 600, increment = 10, start ratio = 0.25 and ratio increment = 0.25
eF.plotGraph4(grahamscan, 600, 10, 0.25, 0.25)

In [ ]:
#Chan's Algorithm

#maxn = 600, increment = 10, start ratio = 0.25 and ratio increment = 0.25
eF.plotGraph4(chen, 600, 10 , 0.25, 0.25)